# Merging datasets

In [307]:
import pandas as pd

In [308]:
path_mercado = 'C:/workspace/Desafio Grão Direto IA/Data/mercado-desafio-tratado.xlsx'
path_transacoes = 'C:/workspace/Desafio Grão Direto IA/Data/transações-desafio-tratado.xlsx'

mercado = pd.read_excel(path_mercado)
transacoes =  pd.read_excel(path_transacoes)

In [309]:
model_df = pd.merge(transacoes,mercado,on='Date',how='left')

In [310]:
model_df

,Date,Seller ID,Last Transaction,Last Week,Last Month,Day of Week,Transactioned,Price,CBOT,Close,Change,Percentage Change
0,2024-01-30,100000060,0,0,0,1,0,97.111044,1215.886963,4.950200,False,0.000000
1,2024-01-31,100000060,0,0,0,2,0,98.585604,1222.299024,4.950300,True,0.002023
2,2024-02-01,100000060,0,0,0,3,0,96.726709,1209.233105,4.951900,True,0.032307
3,2024-02-02,100000060,0,0,0,4,0,97.929889,1199.996115,4.915000,False,-0.750764
4,2024-02-03,100000060,0,0,0,5,0,97.929889,1199.996115,4.915000,False,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
624658,2024-11-01,100000314,0,0,0,4,0,117.601042,1012.899216,5.789400,True,0.468098
624659,2024-11-02,100000314,0,0,0,5,0,117.601042,1012.899216,5.789400,False,0.000000
624660,2024-11-03,100000314,0,0,0,6,0,117.601042,1012.899216,5.789400,False,0.000000
624661,2024-11-04,100000314,0,0,0,0,0,115.506245,1020.956401,5.677526,False,-1.970473


Separando o dataset com temporal_train_test

In [311]:
from sktime.forecasting.model_selection import temporal_train_test_split

In [312]:
seller_id = 100000060
time_serie = model_df[model_df['Seller ID'] == seller_id]
time_serie.drop(columns=['Seller ID'],inplace=True)
time_serie

C:\Users\Miguel Mendes\AppData\Local\Temp\ipykernel_10780\72687540.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_serie.drop(columns=['Seller ID'],inplace=True)


,Date,Last Transaction,Last Week,Last Month,Day of Week,Transactioned,Price,CBOT,Close,Change,Percentage Change
0,2024-01-30,0,0,0,1,0,97.111044,1215.886963,4.950200,False,0.000000
1,2024-01-31,0,0,0,2,0,98.585604,1222.299024,4.950300,True,0.002023
2,2024-02-01,0,0,0,3,0,96.726709,1209.233105,4.951900,True,0.032307
3,2024-02-02,0,0,0,4,0,97.929889,1199.996115,4.915000,False,-0.750764
4,2024-02-03,0,0,0,5,0,97.929889,1199.996115,4.915000,False,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
276,2024-11-01,53,0,0,4,0,117.601042,1012.899216,5.789400,True,0.468098
277,2024-11-02,54,0,0,5,0,117.601042,1012.899216,5.789400,False,0.000000
278,2024-11-03,55,0,0,6,0,117.601042,1012.899216,5.789400,False,0.000000
279,2024-11-04,0,1,1,0,1,115.506245,1020.956401,5.677526,False,-1.970473


In [313]:
# time_serie['year'] = time_serie['Date'].dt.year
# time_serie['month'] = time_serie['Date'].dt.month
# time_serie['day'] = time_serie['Date'].dt.day
# time_serie

In [314]:
time_serie.set_index('Date',inplace=True)
target = time_serie.pop('Transactioned')
time_serie

,Last Transaction,Last Week,Last Month,Day of Week,Price,CBOT,Close,Change,Percentage Change
Date,,,,,,,,,
2024-01-30,0,0,0,1,97.111044,1215.886963,4.950200,False,0.000000
2024-01-31,0,0,0,2,98.585604,1222.299024,4.950300,True,0.002023
2024-02-01,0,0,0,3,96.726709,1209.233105,4.951900,True,0.032307
2024-02-02,0,0,0,4,97.929889,1199.996115,4.915000,False,-0.750764
2024-02-03,0,0,0,5,97.929889,1199.996115,4.915000,False,0.000000
...,...,...,...,...,...,...,...,...,...
2024-11-01,53,0,0,4,117.601042,1012.899216,5.789400,True,0.468098
2024-11-02,54,0,0,5,117.601042,1012.899216,5.789400,False,0.000000
2024-11-03,55,0,0,6,117.601042,1012.899216,5.789400,False,0.000000


In [315]:
dates_to_pred = '2024-11-04'

In [316]:
y_train, y_test, X_train, X_test = temporal_train_test_split(target,time_serie,test_size=2)

TypeError: temporal_train_test_split() got an unexpected keyword argument 'stratify'

Creating forecasting

In [300]:
from sktime.forecasting.base import ForecastingHorizon

fh = ForecastingHorizon([1,2],is_relative=True)

In [301]:
from xgboost import XGBRegressor

# Configurando o regressor
xgb_model = XGBRegressor(
    n_estimators=100,  # Número de árvores no ensemble
    learning_rate=0.1, # Taxa de aprendizado
    max_depth=5,       # Profundidade máxima da árvore
    subsample=0.8,     # Subamostragem das amostras de treino
    colsample_bytree=0.8, # Subamostragem de features por árvore
    random_state=42
)

In [302]:
from sktime.forecasting.compose import make_reduction

forecaster = make_reduction(xgb_model, window_length=52, strategy="recursive")

In [ ]:
forecaster.fit(y=y_train,X=X_train)

In [ ]:
y_pred = forecaster.predict(fh=fh,X=X_test)
y_pred

In [ ]:
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error

print('MAPE: %.4f' % mean_absolute_percentage_error(y_test, y_pred, symmetric=False))

In [ ]:
print(target.value_counts(normalize=True))
import matplotlib.pyplot as plt

target.value_counts().plot(kind='bar')
plt.title('Distribuição das Classes no Target')
plt.xlabel('Classe')
plt.ylabel('Frequência')
plt.show()
